In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def mse(true, pred):
    return np.mean((pred - true) ** 2)

def mse_derivative(true, pred):
    return 2 * (pred - true) / true.size

In [19]:
def standarize(data, train_mean, std):
    return (data - train_mean) / std

def min_max(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [20]:
class MLP:
    def __init__(self, input_dimension, hidden_dimension, output_dimension, learning_rate):
        self.learning_rate = learning_rate

        self.weight_input_hidden = np.random.randn(input_dimension, hidden_dimension) * 0.01
        self.weight_hidden_output = np.random.randn(hidden_dimension, output_dimension) * 0.01

        self.bias_hidden = np.zeros((1, hidden_dimension))
        self.bias_output = np.zeros((1, output_dimension))

    def forward_propagation(self, x):
        self.x = x
        self.hidden_layer_input = np.dot(x, self.weight_input_hidden) + self.bias_hidden
        self.hidden_layer_output = relu(self.hidden_layer_input)
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weight_hidden_output) + self.bias_output
        self.output = self.output_layer_input
        return self.output
    
    def backward_propagation(self, y_true, y_pred):
        d_output_layer_input = mse_derivative(y_true, y_pred)

        d_weight_hidden_output = np.dot(self.hidden_layer_output.T, d_output_layer_input)
        d_bias_output = np.sum(d_output_layer_input, axis=0, keepdims=True)

        d_hidden_layer_output = np.dot(d_output_layer_input, self.weight_hidden_output.T)
        d_hidden_layer_input = d_hidden_layer_output * relu_derivative(self.hidden_layer_input)

        d_weight_input_hidden = np.dot(self.x.T, d_hidden_layer_input)
        d_bias_hidden = np.sum(d_hidden_layer_input, axis=0, keepdims=True)

        self.weight_input_hidden -= self.learning_rate * d_weight_input_hidden
        self.weight_hidden_output -= self.learning_rate * d_weight_hidden_output
        self.bias_hidden -= self.learning_rate * d_bias_hidden
        self.bias_output -= self.learning_rate * d_bias_output

    def train(self, x, y, epochs):
        loss_list = []
        for epoch in range(epochs):
            y_pred = self.forward_propagation(x)
            loss = mse(y, y_pred)
            loss_list.append(loss)
            self.backward_propagation(y, y_pred)     
            if (epoch + 1) % 100 == 0:
                print(f"Epoch {epoch + 1}, Loss: {loss}")
        return loss_list, y_pred
    
    def evaluate(self, x, y):
        y_pred = self.forward_propagation(x)
        return y_pred

In [21]:
file_path = '../../Datasets/조업편차분석.csv'
data = pd.read_csv(file_path)

data['Origin'] = data.index
shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=True)

train_size = int(len(shuffled_data) * 0.8)
train_data, test_data = shuffled_data.iloc[:train_size], shuffled_data.iloc[train_size:]

In [22]:
x_train, y_train = train_data.drop(columns=['No.', 'A1', 'Origin'], axis=1), train_data['A1']

x_train_mean = np.mean(x_train, axis=0)
x_train_std = np.std(x_train, axis=0)
y_train_mean = np.mean(y_train, axis=0)
y_train_std = np.std(y_train, axis=0)

x_train = standarize(x_train, x_train_mean, x_train_std).to_numpy()
y_train = standarize(y_train, y_train_mean, y_train_std).to_numpy().reshape(-1, 1)

# x_train = min_max(x_train).to_numpy()
# y_train = min_max(y_train).to_numpy().reshape(-1, 1)

Epoch 100, Loss: 0.9866349779714318
Epoch 200, Loss: 0.9635930754259823
Epoch 300, Loss: 0.9131894499250345
Epoch 400, Loss: 0.8069801316740027
Epoch 500, Loss: 0.6213408127741155
Epoch 600, Loss: 0.39418936449229275
Epoch 700, Loss: 0.22329172043012843
Epoch 800, Loss: 0.13823373907826733
Epoch 900, Loss: 0.09937013054792337
Epoch 1000, Loss: 0.07798528752735404


In [23]:
input_dimension = x_train.shape[1]
hidden_dimension = 64
output_dimension = y_train.shape[1]

mlp_model = MLP(
    input_dimension=input_dimension, 
    hidden_dimension=hidden_dimension, 
    output_dimension=output_dimension, 
    learning_rate=0.001)
loss_list, y_pred_train = mlp_model.train(x_train, y_train, epochs=1000)

plt.figure(figsize=(10, 6))
plt.plot(loss_list)
plt.show()

In [24]:
test_data_sorted = test_data.sort_values(by='Origin').reset_index(drop=True)

x_test_sorted, y_test_sorted = test_data_sorted.drop(columns=['No.', 'A1', 'Origin'], axis=1), test_data_sorted['A1']
x_test_sorted = standarize(x_test_sorted, x_train_mean, x_train_std).to_numpy()
y_test_sorted = standarize(y_test_sorted, y_train_mean, y_train_std).to_numpy().reshape(-1, 1)

# x_test_sorted = min_max(x_test_sorted).to_numpy()
# y_test_sorted = min_max(y_test_sorted).to_numpy().reshape(-1, 1)

y_test_pred = mlp_model.evaluate(x_test_sorted, y_test_sorted)

plt.figure(figsize=(50, 6))
plt.plot(y_test_sorted, marker="o")
plt.plot(y_test_pred, marker="x")
plt.show()

In [25]:
def evaluate_metrics(y_true, y_pred):
    mae = np.mean(np.abs(y_true - y_pred))
    mse_value = mse(y_true, y_pred)
    rmse = np.sqrt(mse_value)
    ss_total = np.sum((y_true - np.mean(y_true)) ** 2)
    ss_residual = np.sum((y_true - y_pred) ** 2)
    r_square = 1 - (ss_residual / ss_total)
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse_value:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R-square: {r_square:.4f}")

evaluate_metrics(y_test_sorted, y_test_pred)

MAE: 0.2085
MSE: 0.0831
RMSE: 0.2882
R-square: 0.9257
